In [0]:
# Import libraries
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [0]:
# Load the league ranking data from the Gold table
df_football_ranking = spark.read.table("football_matches_catalog.gold.football_matches_ranking")
display(df_football_ranking)

In [0]:
# Find the champion for each season order by season and drop team position
df_champions_by_season = (df_football_ranking
                          .filter(col("TeamPosition") == 1)
                          .drop("TeamPosition")
                          .orderBy(col("Season"))
                        )


In [0]:
# Display the champion for each season
display(df_champions_by_season)

In [0]:
# Count how many titles each team has won
df_titles_by_team = (df_football_ranking
                     .filter(col("TeamPosition") == 1)
                     .groupBy(col("Team"))
                     .count()
                     .orderBy(col("count").desc())
                     .withColumnRenamed("count", "Titles")
                     .drop("TeamPosition")
                     )


In [0]:
# Display the number of titles
display(df_titles_by_team)

In [0]:

# Find the best attacking team (most goals scored) for each season
attack_rank_window = Window.partitionBy("Season").orderBy(F.desc("GoalsScored"))

df_best_attack_by_season = (
    df_football_ranking
        .withColumn("AttackRank", F.row_number().over(attack_rank_window))
        .filter(F.col("AttackRank") == 1)
        .select("Season", "Team", "GoalsScored")
        .orderBy("Season")
)

In [0]:
# Display the best attacking team each season
display(df_best_attack_by_season)